In [4]:
#Run for downloading python Modules
"""
cut the below and paste here to ignore
"""
!pip install datetime
!pip install bs4
!pip install requests
!pip install geograpy3
!pip install nltk
!pip install openpyxl
!pip install geopandas
!pip install folium
!pip install spacy
!pip install pycountry
!pip install html2text
!pip install pandas
!pip install matplotlib
!python -m spacy download pt_core_news_sm
!pip install contextily

     |████████████████████████████████| 60 kB 213 kB/s eta 0:00:01
     |████████████████████████████████| 235 kB 720 kB/s eta 0:00:01


In [7]:
from datetime import date,timedelta
import requests
import nltk
from bs4 import BeautifulSoup
import geograpy 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import openpyxl
import html2text
import pycountry
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
import pt_core_news_sm
import pandas as pd #used to read in the revenue file 
import matplotlib.pyplot as plt #for plotting#to read in shape file and provides high #level interface with #matplotlib library for making maps
import geopandas as gpd#import locally saved file
import folium
from geopy.geocoders import Nominatim

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
def web_travel():
    file=openpyxl.Workbook()
    std=file['Sheet']
    std.append(['url','flag','Country','News'])
    h = html2text.HTML2Text()
    # Ignore converting links from HTML
    h.ignore_links = True
    today = date.today()
    yesterday = today - timedelta(days=1)
    print("Yesterday's date:", yesterday)
    try: 
        from googlesearch import search 
    except ImportError:  
        print("No module named 'google' found") 
    keywords=['new pandemic','new epidemic','new outbreak','new lockdown','new quarantine','deaths','new viral outbreak']
    article=[]
    for query in keywords:  
        for j in search(query+" after:"+str(yesterday), tld="com", num=100, stop=5, pause=2): 
            if j.find('corona')==-1 and j.find('covid')==-1:
                print(j)
                article.append(j)
    a=[]
    raspin=1
    for it in article:
        try:
            #print(i)
            flag=0
            url = it#"https://health.economictimes.indiatimes.com/news/industry/china-study-warns-of-possible-new-pandemic-virus-from-pigs/76721325"
            places = geograpy.get_place_context(url=url) 
            Location=[]
            page=requests.get(url)
            soup = BeautifulSoup.get_text(BeautifulSoup(page.text, 'html.parser'))
            viral_data=soup.find("virus")
            virus=soup[viral_data-10:viral_data+5]
            #print(virus)
            places = geograpy.get_place_context(url=url) 
            max=0
            for i in places.country_mentions:
                try:
                    if pycountry.countries.lookup(i[0]):
                        Location.append(pycountry.countries.lookup(i[0]).name)
                        if max< i[1]:
                            max=i[1]
                            max_location=i[0]
                except:
                    pass
            #nlp = pt_core_news_sm.load()
            if soup.find('Denied')!=-1 or soup.find('connect to the server')!=-1 :
                #print("Suka")
                continue
            nlp=spacy.load('en_core_web_sm')#en_core_web_sm.load()
            f=soup
            text=f
            #text = f.replace("  ","")
            #text=f.replace("\n"," ")
            doc = nlp(text)
            entities=[]
            something=[]
            something_significant=""
            matches = ["affect", "dea", "case","patients"]
            for i in doc.ents:
                if i.label_=='CARDINAL':
                    entities.append((i.text,text[i.start_char-10:i.end_char+10], i.label_))#, i.label))
                    something.append(text[i.start_char-30:i.end_char+30])
                    if any(x in text[i.start_char:i.end_char+20] for x in matches):
                        #print(text[i.start_char:i.end_char+20])
                        #print(text[i.start_char:i.end_char+20].find("dea"))
                        #print(matches)
                        if text[i.start_char:i.end_char+20].find("affect")!=-1:
                            matches.remove("affect")
                        if text[i.start_char:i.end_char+20].find("dea")!=-1:
                            matches.remove("dea")
                        if text[i.start_char:i.end_char+20].find("case")!=-1:
                            matches.remove("case")
                        if text[i.start_char:i.end_char+20].find("patients")!=-1:
                            matches.remove("patients")
                        if matches==[]:
                            continue
                        else:
                            pass
                        '''
                        for num in range(i.start_char,i.start_char-100,-1):
                            if text[i.start_char-num]=="." or text[i.start_char-num-1:i.start_char-num]=="  " :
                                break
                        for num2 in range(i.end_char,i.end_char+100,1):
                            try:
                                if text[i.end_char+num2]==".":
                                    break
                            except:
                                break
                        '''
                        something_significant=something_significant+text[i.start_char:i.end_char+35]+". "
                        #something_significant.append(text[i.start_char:i.end_char+100])
                else: 
                    pass 
            corpus = [sent.text.lower() for sent in doc.sents ]
            cv = CountVectorizer(stop_words=list(STOP_WORDS))   
            cv_fit=cv.fit_transform(corpus)    
            word_list = cv.get_feature_names();    
            count_list = cv_fit.toarray().sum(axis=0)
            word_frequency = dict(zip(word_list,count_list))
            val=sorted(word_frequency.values())
            higher_word_frequencies = [word for word,freq in word_frequency.items() if freq in val[-3:]]
            #print("\nWords with higher frequencies: ", higher_word_frequencies)# gets relative frequency of words
            higher_frequency = val[-1]
            for word in word_frequency.keys():  
                word_frequency[word] = (word_frequency[word]/higher_frequency)
            sentence_rank={}
            for sent in doc.sents:
                for word in sent :       
                    if word.text.lower() in word_frequency.keys():            
                        if sent in sentence_rank.keys():
                            sentence_rank[sent]+=word_frequency[word.text.lower()]
                        else:
                            sentence_rank[sent]=word_frequency[word.text.lower()]
            top_sentences=(sorted(sentence_rank.values())[::-1])
            top_sent=top_sentences[:3]
            summary=[]
            for sent,strength in sentence_rank.items():  
                if strength in top_sent:
                    summary.append(sent)
                else:
                    continue 
            #print(url,max_location,something_significant,sep=" ",end="\n################3\n")
            summary=""
            for i in summary:
                print(i,end=" ")
                summary=summary+" "+i
            #print("\nLOCATION####################\n")
            #print(Location,end="\nSomething####################\n")
            #print(something,end="\nEntities####################\n")
            #print(entities,end="\n###########################################################################3\n ")
            #strings=strings+"\n####################\n"+str(Location)+"\n####################\n"+str(entities)+"\n############"+str(something)+"###############################################################3\n "
            strings=url+"\n"+str(max_location)+"\n"+str(something_significant)+"\n##########\n"
            if len(something_significant)>1 and len(virus)!=0:
                with open("ReportFile.txt", "a") as file1: 
                    file1.write(strings)
                    std.append([url,1,str(max_location),str(max_location)+" "+virus+" "+str(something_significant)+" "+url,summary])
                    raspin+=1
        except:
            pass
    file.save("ReportFile.xlsx")
    flag=1
    return flag

In [3]:
flag=web_travel()
geolocator = Nominatim(user_agent="infinitum_circa")
revenue = pd.read_excel('ReportFile.xlsx')#import world shape map externally, no need to have a saved file
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#view first 5 rows
revenue.head()#inspect count
revenue.info()#view first 5 rows
world.head()#inspect count
world.info()
#rge both data sets using country code/iso_a3 as unique identifiers

for_plotting = world.merge(revenue, left_on = 'name', right_on = 'Country')#check the occurrence
for_plotting.info() 
#plot merged file. use dropna to remove any country with no revenue value
#ax = for_plotting.dropna().plot(column='New Cases', cmap = 'YlGnBu', figsize=(15,9))#,scheme='quantiles', k=3, legend = True)
#ax = for_plotting.plot(column='flag', cmap = 'YlOrRd', figsize=(15,9),scheme='quantiles', k=100, legend = False)#,colorbar= True)
#add title to the map
#ax.set_title('Total New Case by country', fontdict= {'fontsize':25})#remove axes
#ax.set_axis_off()

import contextily as ctx
lat=[]
long=[]
country=[]
summary=[]
for i in range(len(revenue['Country'])):
    if revenue['Country'][i] in country:
        pass
    else:
        country.append(revenue['Country'][i])
        location = geolocator.geocode(revenue['Country'][i])
        lat.append(location.latitude)
        long.append(location.longitude)
        summary.append(revenue['News'][i])
m = folium.Map()
m.choropleth(world, data=for_plotting, key_on='feature.properties.name',
             columns=['name','flag'],fill_color='YlOrRd', nan_fill_color='white',figsize=(15,9))#,scheme='quantiles', k=100, legend = False)
for i in range(0,len(summary)):
    folium.Marker([lat[i], long[i]], popup=summary[i]).add_to(m)
m

Yesterday's date: 2020-08-29
https://www.bbc.com/news/world-51235105
https://apnews.com/VirusOutbreak
https://apnews.com/UnderstandingtheOutbreak
https://www.cdc.gov.tw/En
https://www.cdc.gov.tw/En/Bulletin/Detail/pevkg6JalT40I4uB6wAWcg?typeid=158
https://www.cdc.gov.tw/En/Bulletin/Detail/YS_6xZ01kBHnOpamUwyEgA?typeid=158
https://www.cdc.gov.tw/En/Bulletin/List/7tUXjTBf6paRvrhEl-mrPg
https://www.cdc.gov.tw/En/Category/Page/yZOu-4cGeu77HDyzE0ojqg
https://www.bbc.com/news/world-51235105
https://apnews.com/VirusOutbreak
https://apnews.com/UnderstandingtheOutbreak
https://apnews.com/33ce920720ee71dbabcd8db757b8dfd3
https://apnews.com/OneGoodThing
https://indianexpress.com/article/india/india-unlock-4-0-guidelines-6575245/
https://www.livemint.com/news/india/unlock-4-lockdown-in-containment-zones-extended-till-30-sept-metro-can-resume-11598711098140.html
https://www.jansatta.com/national/unlock-4-0-guidelines-rules-live-news-in-hindi-india-lockdown-unlock-4-0-new-guidelines-latest-news-utta

/home/bhavya_bhardwaj/Downloads/112358/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
